In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Audio

%matplotlib inline

## Generating a Signal

First, let's generate a signal with frequencies 256 Hz and 554 Hz in equal parts.

In [ ]:
fs = 8000
t = np.arange(0, 5, step=1. / fs)
x = np.sin(2 * np.pi * 256 * t) + np.sin(2 * np.pi * 554 * t)

plt.plot(t, x)
plt.xlim(0, .1)

You can use the `Audio()` function inside `IPython.display` to listen to any signal. Don't forget to specify the sampling rate.

In [ ]:
Audio(x, rate=fs)

## Frequency Analysis of the Signal

`np.fft.fft` is an implementation of the Fast Fourier Transform (FFT), which is an efficient algorithm that calculates the Discrete Fourier Transform (DFT).

Given a time-domain signal of length $N$, the first $\frac{N}{2}$ samples represent the values of the DFT at equally spaced frequencies in $[0, f_s / 2)$ and the last $\frac{N}{2}$ samples represent the values of the DFT at equally spaced frequencies in $[-f_s / 2, 0)$.

If the time-domain signal is real, then the DFT values at negative frequencies must be the complex conjugate of the DFT values at positive frequencies. So all of the information is in the first half of the DFT.

In [ ]:
x_hat = np.fft.fft(x)
x_hat

Since the values of the DFT are complex numbers, we often look at the magnitude or the power of these numbers.

In [ ]:
x_hat_pow = np.abs(x_hat) ** 2
plt.plot(x_hat_pow)

Let's plot this with a proper frequency axis.

In [ ]:
N = len(x)
f = np.arange(0, fs / 2, step=fs / N)
plt.plot(f, x_hat_pow[:(N // 2)])

Now let's add the negative frequencies as well.

In [ ]:
N = len(x)
f = np.arange(0, fs / 2, step=fs / N)
plt.plot(f, x_hat_pow[:(N // 2)])

f_neg = np.arange(-fs / 2, 0, step=fs / N)
plt.plot(f_neg, x_hat_pow[(N // 2):])

## Designing the Filter

Let's design a filter that keeps the 256 Hz sinusoid, but removes the 554 Hz sinusoid.

We first design the filter that we want in the frequency domain. A lowpass filter with a cutoff frequency of 400 Hz should do the trick.

First, let's find the frequency response at positive frequencies.

In [ ]:
f_pos = np.arange(0, fs / 2, step=fs / N)
h_hat_pos = 1. * (f_pos <= 400)
plt.plot(f_pos, h_hat_pos)
plt.ylim(-.1, 1.1)

Now let's find the frequency response at negative frequencies.

In [ ]:
f_neg = np.arange(-fs / 2, 0, step=fs / N)
h_hat_neg = 1. * (f_neg >= -400)
plt.plot(f_neg, h_hat_neg)
plt.ylim(-.1, 1.1)

Finally, let's combine the frequency response at positive and negative frequencies. Remember that the negative frequencies go at the _end_ after the positive frequencies.

In [ ]:
f = np.concatenate([f_pos, f_neg])
h_hat = np.concatenate([h_hat_pos, h_hat_neg])

Now let's take the _inverse_ DFT to find the impulse response of this filter.

In [ ]:
h = np.real(np.fft.ifft(h_hat))
plt.plot(h)

Let's truncate and put this impulse response in the right order.

In [ ]:
h_trunc = np.concatenate([h[-1000:], h[:1000]])
plt.plot(h_trunc)

## Applying the Filter

Now we apply the filter by convolving the impulse response with the signal.

In [ ]:
y = np.convolve(x, h_trunc)
y = y[1:] # remove the first sample (since there are an odd number of samples)

In [ ]:
N = len(y)
t = np.arange(0, N / fs, step=1. / fs)
plt.plot(t, y)
plt.xlim(.3, .4)

Let's do a frequency analysis of this signal.

In [ ]:
f_pos = np.arange(0, fs / 2, step=fs / N)
y_hat = np.fft.fft(y)
y_hat_pow = np.abs(y_hat) ** 2

plt.plot(f_pos, y_hat_pow[:(N // 2)])

Finally, let's listen to the filtered signal.

In [ ]:
Audio(y, rate=fs)